In [ ]:
# The following codes to import, clean, and process Sentinel-2 data using VSI and AWS were written by Lilly Jones and Erick Verleye; edited by Ty Tuff, pseudocode by Cibele Amaral. 

#imports

import os
import glob
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from osgeo import gdal, gdalconst
import numpy as np
import rasterio
import matplotlib.pyplot as plt

In [ ]:

# 1. Download images using Sentinel-2 API

# download Sentinel-2 data by date and bounding box

def download_sentinel_data(username, password, start_date, end_date, bounding_box, output_directory):
    """
    Downloads Sentinel-2 data using the Copernicus API Hub.

    Args:
        username (str): Your Copernicus Open Access Hub username.
        password (str): Your Copernicus Open Access Hub password.
        start_date (str): Start date in 'YYYYMMDD' format (e.g., '20220101').
        end_date (str): End date in 'YYYYMMDD' format (e.g., '20220131').
        bounding_box (list): Bounding box coordinates in the format [lon_min, lat_min, lon_max, lat_max].
        output_directory (str): Directory where downloaded data will be stored.

    Returns:
        None
    """
    # Initialize the Sentinel API with your credentials
    api = SentinelAPI(username, password, 'https://apihub.copernicus.eu/dhus')

    # Convert the bounding box to Well-Known Text (WKT) format
    footprint = geojson_to_wkt({
        "type": "Polygon",
        "coordinates": [bounding_box]
    })

    # Search for Sentinel-2 products within the specified date range and bounding box
    products = api.query(footprint,
                         date=(start_date, end_date),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))  # You can adjust the cloud cover percentage as needed

    # Download the products
    for product_id, product_info in products.items():
        print(f"Downloading product {product_id}...")
        api.download(product_id, directory_path=output_directory)

if __name__ == "__main__":
    # Set your Copernicus Open Access Hub credentials
    username = "your_username"
    password = "your_password"

    # Define the date range and bounding box for your area of interest
    start_date = "20220101"
    end_date = "20220131"
    bounding_box = [lon_min, lat_min, lon_max, lat_max]  # Replace with your bounding box coordinates
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the download_sentinel_data function
    download_sentinel_data(username, password, start_date, end_date, bounding_box, output_directory)

In [ ]:
# 2. Resample bands to 10-m resolution
- Select bands 2 to 8A and 11 to 12 (10 bands total)
- Resample the 20-m resolution bands to 10-m resolution 
- Stack the 10 bands into one raster
- Gdal (https://gdal.org/api/python_bindings.html) does everything listed above

import gdal
from gdalconst import *

# Input file paths
input_file_20m = 'path_to_input_20m_image.tif'
input_file_10m = 'path_to_input_10m_image.tif'

# Output file path
output_file = 'path_to_output_image.tif'

# Open the 20m resolution raster
ds_20m = gdal.Open(input_file_20m, GA_ReadOnly)

# Resample the 20m resolution bands to 10m resolution
ds_10m = gdal.Warp('', ds_20m, xRes=10, yRes=10)

# Select bands 2 to 8A and 11 to 12 (10 bands total)
band_selection = [2, 3, 4, 5, 6, 7, 8, 11, 12]

# Stack the selected bands into one raster
output_ds = gdal.BuildVRT('', [ds_10m.GetRasterBand(band) for band in band_selection])

# Write the stacked raster to an output file
gdal.Translate(output_file, output_ds, format='GTiff')

# Clean up
ds_20m = None
ds_10m = None
output_ds = None

In [ ]:
# Original AWS code by Erick Verleye, ESIIL Software Developer 2023-08-08, lightly edited by Lilly Jones for SCE project
# Code downloads Sentinel Level-1 C data from S3 with Python for a given latitude, longitude, and date range.
# An AWS account is needed and will be charged (a small amount) for any data downloaded.

#imports

import json

import multiprocessing as mp
import os

from argparse import Namespace
from datetime import datetime, timedelta
from typing import List, Tuple, Union

import boto3
import geojson
import tqdm
from shapely.geometry import Polygon

# Use boto3 to create a connection to the AWS account. Initialization of the boto3.session object changes depending on the environment you are running this code in:

# Using the AWS CLI from a personal computer to log in, the profile name will typically be default.
# Using a federated access login, the profile name will typically be the name of the software.
# Using S3 configured IAM profile, no profile name is required.
# Each of these arguments, if applicable, can be found in the aws credentials file typically found at ~/.aws/credentials


# Replace with your AWS profile name if applicable
profile = 'default'

session = boto3.Session(profile_name=profile)

# AWS CLI
session = boto3.Session(profile_name='default')

# Federated login (saml2aws example)
session = boto3.Session(profile_name='saml')

# S3 configured IAM profile on EC2 instance
session = boto3.Session()

# Define constants

SENTINEL_2_BUCKET_NAME = 'insert_name_of_S3_bucket'  # Name of the s3 bucket on AWS hosting the sentinel-2 data


# find_overlapping_mgrs Sentinel-2 data is organized depending on which Military Grid Reference System (MGRS) square that it belongs to. This function converts a bounding box defined in    # lat/lon as [min_lon, min_lat, max_lon, max_lat] to the military grid squares that is overlaps. More information on # # the MGRS can be found at                                                       # https://www.maptools.com/tutorials/mgrs/quick_guide. NOTE: You will have to download the mgrs_lookup.geojson file from # # # # # https://github.com/CU-ESIIL/data-                        # library/blob/main/docs/remote_sensing/sentinel2_aws/mgrs_lookup.geojson and place it into the working directory that # # this code is being run from.

def find_overlapping_mgrs(bounds: List[float]) -> List[str]:
    """
    Files in the Sinergise Sentinel2 S3 bucket are organized by which military grid they overlap. Thus, the
    military grids that overlap the input bounding box defined in lat / lon must be found. A lookup table that
    includes each grid name and its geometry is used to find the overlapping grids.
    Args:
        bounds (list): Bounding box definition as [min_lon, min_lat, max_lon, max_lat]
    """
    print('Finding overlapping tiles... ')
    input_bounds = Polygon([(bounds[0], bounds[1]), (bounds[2], bounds[1]), (bounds[2], bounds[3]),
                            (bounds[0], bounds[3]), (bounds[0], bounds[1])])
    with open('mgrs_lookup.geojson', 'r') as f:
        ft = geojson.load(f)
        return [i['properties']['mgs'] for i in ft[1:] if
                input_bounds.intersects(Polygon(i['geometry']['coordinates'][0]))]

# find_available_files finds the set of available files in the s3 bucket given a date range, lat/lon bounds, and list of bands.

def find_available_files(s3_client, bounds: List[float], start_date: datetime, end_date: datetime,
                         bands: List[str]) -> List[Tuple[str, str]]:
    """
    Given a bounding box and start / end date, finds which files are available on the bucket and
    meet the search criteria
    Args:
        bounds (list): Lower left and top right corner of bounding box defined in
        lat / lon [min_lon, min_lat, max_lon, max_lat]
        start_date (str): Beginning of requested data creation date YYYY-MM-DD
        end_date (str): End of requested data creation date YYYY-MM-DD
    """
    date_paths = []
    ref_date = start_date
    while ref_date <= end_date:
        tt = ref_date.timetuple()
        date_paths.append(f'/{tt.tm_year}/{tt.tm_mon}/{tt.tm_mday}/')
        ref_date = ref_date + timedelta(days=1)

    info = []
    mgrs_grids = find_overlapping_mgrs(bounds)
    for grid_string in mgrs_grids:
        utm_code = grid_string[:2]
        latitude_band = grid_string[2]
        square = grid_string[3:5]
        grid = f'tiles/{utm_code}/{latitude_band}/{square}'
        response = s3_client.list_objects_v2(
            Bucket=SENTINEL_2_BUCKET_NAME,
            Prefix=grid + '/',
            MaxKeys=300,
            RequestPayer='requester'
        )
        if 'Contents' not in list(response.keys()):
            continue

        for date in date_paths:
            response = s3_client.list_objects_v2(
                Bucket=SENTINEL_2_BUCKET_NAME,
                Prefix=grid + date + '0/',  # '0/' is for the sequence, which in most cases will be 0
                MaxKeys=100,
                RequestPayer='requester'
            )
            if 'Contents' in list(response.keys()):
                info += [
                    (v['Key'], v['Size']) for v in response['Contents'] if
                    any([band + '.jp2' in v['Key'] for band in bands]) or 'MSK_CLOUDS_B00.gml' in v['Key']
                ]

    return info

In [ ]:
# This function is designed to download the files in parallel, so a download_task function is defined as well. 
# Note that multiprocessing will not work as implemented here in iPython (Jupyter # # notebooks, etc.) and so that block of code is commented out. 
# If you are running this in a different environment and would like to download in parallel, un-comment this block and comment the sequential block below.


def download_task(namespace: Namespace) -> None:
    """
    Downloads a single file from the indicated s3 bucket. This function is intended to be spawned in parallel from the
    parent process.
    Args:
        namespace (Namespace): Contains the bucket name, s3 file name, and destination required for s3 download request.
        Each value in the namespace must be a pickle-izable type (i.e. str).
    """
    session = boto3.Session(profile_name=namespace.profile_name)
    s3 = session.client('s3')
    s3.download_file(namespace.bucket_name, namespace.available_file,
                     namespace.dest,
                     ExtraArgs={'RequestPayer': 'requester'}
                     )


def download(session, bounds: List[float], start_date: datetime, end_date: datetime,
             bands: List[float], out_dir: str, buffer: float = None) -> None:
    """
    Downloads a list of .jp2 files from the Sinergise Sentinel2 LC1 bucket given a bounding box defined in lat/long, a buffer in meters, and a start and end date. Only Bands 2-4 are requested.
     Args:
         bounds (list): Bounding box defined in lat / lon [min_lon, min_lat, max_lon, max_lat]
         buffer (float): Amount by which to extend the bounding box by, in meters
         start_date (str): Beginning of requested data creation date YYYY-MM-DD
         end_date (str): End of requested data creation date YYYY-MM-DD
         bands (list): The bands to download for each file. Ex. ['B02', 'B03', 'B04', 'B08'] for R, G, B, and
         near wave IR, respectively
         out_dir (str): Path to directory where downloaded files will be written to
    """
    # Convert the buffer from meters to degrees lat/long at the equator
    if buffer is not None:
        buffer /= 111000

        # Adjust the bounding box to include the buffer (subtract from min lat/long values, add to max lat/long values)
        bounds[0] -= buffer
        bounds[1] -= buffer
        bounds[2] += buffer
        bounds[3] += buffer

    s3_client = session.client('s3')
    available_files = find_available_files(s3_client, bounds, start_date, end_date, bands)

    args = []
    total_data = 0
    for file_info in available_files:
        file_path = file_info[0]
        if '/preview/' in file_path:
            continue

        created_file_path = os.path.join(out_dir, file_path.replace('_qi_', '').replace('/', '_').replace('tiles_', ''))

        # Skip if file is already local
        if os.path.exists(created_file_path):
            continue

        total_data += file_info[1]

        args.append(Namespace(available_file=file_path, bucket_name=SENTINEL_2_BUCKET_NAME, profile_name=session.profile_name,
                              dest=created_file_path))

    total_data /= 1E9
    print(f'Found {len(args)} files for download. Total size of files is'
          f' {round(total_data, 2)}GB and estimated cost will be ${round(0.09 * total_data, 2)}'
          )

    # For multiprocessing when being run in iPython (Jupyter notebook, etc.)
    # with mp.Pool(mp.cpu_count() - 1) as pool:
        # for _ in tqdm.tqdm(pool.imap_unordered(download_task, args), total=len(args)):
            # pass

    # Sequential download for use in iPython (Jupyter notebooks, etc.)
    for _ in tqdm.tqdm(args, total=len(args)):
        download_task(args)

# Once all of the above functions and variables are defined, download for a specific time range and region. 
# Make sure the directory corresponding to the out_dir parameter you pass in already exists. If not, create the directory.

# Define the bounding box for California [min_lon, min_lat, max_lon, max_lat]
sce_bounds = [-124.482003, 32.528832, -114.130775, 42.009518]

# Define the output directory
out_dir = 'SCE'
os.makedirs(out_dir, exist_ok=True)


# Download RGB band data for California from a specific date range
download(session=session, bounds=california_bounds, start_date=datetime(YYYY, MM, DD),  # Replace with your date range
         end_date=datetime(YYYY, MM, DD),  # Replace with your date range
         bands=['B02', 'B03', 'B04'], out_dir=out_dir)


# Cloud files will be downloaded alongside the band data.

In [ ]:
# The following cloud probability mask code is by Erick Verleye

# Define constants
MAX_BAND_VAL = 4000  # counts

# Helper function for opening Sentinel-2 jp2 files, with optional slicing

def get_img_from_file(img_path, g_ncols, dtype, row_bound=None):
    img = rasterio.open(img_path, driver='JP2OpenJPEG')
    ncols, nrows = img.meta['width'], img.meta['height']
    assert g_ncols == ncols, f'Imgs have different size ncols: {ncols} neq {g_ncols}'
    if row_bound is None:
        pixels = img.read(1).astype(np.float32)
    else:
        pixels = img.read(
            1,
            window=Window.from_slices(
                slice(row_bound[0], row_bound[1]),
                slice(0, ncols)
            )
        ).astype(dtype)
    return pixels

OR: def get_img_from_file(img_path, g_ncols, dtype, row_bound=None):
    img = rasterio.open(img_path, driver='JP2OpenJPEG')
    ncols, nrows = img.meta['width'], img.meta['height']
    assert g_ncols == ncols, f'Imgs have different size ncols: {ncols} neq {g_ncols}'
    if row_bound is None:
        pixels = img.read(1).astype(np.float32)
    else:
        pixels = img.read(
            1,
            window=Window.from_slices(
                slice(row_bound[0], row_bound[1]),
                slice(0, ncols)
            )
        ).astype(dtype)
    return pixels

# Helper function for reading in cloud file array, with optional slicing
def get_cloud_mask_from_file(cloud_path, crs, transform, shape, row_bound=None):
    # filter out RuntimeWarnings, due to geopandas/fiona read file spam
    # https://stackoverflow.com/questions/64995369/geopandas-warning-on-read-file
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    try:
        cloud_file = gpd.read_file(cloud_path)
        cloud_file.crs = (str(crs))
        # convert the cloud mask data to a raster that has the same shape and transformation as the
        # img raster data
        cloud_img = features.rasterize(
            (
                (g['geometry'], 1) for v, g in cloud_file.iterrows()
            ),
            out_shape=shape,
            transform=transform,
            all_touched=True
        )
        if row_bound is None:
            return np.where(cloud_img == 0, 1, 0)
        return np.where(cloud_img[row_bound[0]:row_bound[1], :] == 0, 1, 0)
    except Exception as e:
        return None

# Function for filtering out cloud pixels

def nan_clouds(pixels, cloud_channels, max_pixel_val: float = MAX_BAND_VAL):
    cp = pixels * cloud_channels
    mask = np.where(np.logical_or(cp == 0, cp > max_pixel_val))
    cp[mask] = np.nan
    return cp

OR: def nan_clouds(pixels, cloud_channels, max_pixel_val: float = MAX_BAND_VAL):
    cp = pixels * cloud_channels
    mask = np.where(np.logical_or(cp == 0, cp > max_pixel_val))
    cp[mask] = np.nan
    return cp

# Function to create cloud-cleaned composite for a single military tile and optical band

def create_cloud_cleaned_composite(in_dir: str, mgrs_tile: str, band: str, out_file: str, num_slices: int = 12) -> None:
    """
    Creates a cloud cleaned composite tif file from a set of sentinel 2 files
    Args:
        in_dir (str): Directory containing all of the date directories which contain the sentinel 2 data
        mgrs_tile (str): The military grid coordinate (35MGR, 36MTV, etc.) to create the composite for
        band (str): The optical band for which to create the composite for (B02, B03, B08) etc.
        num_slices (int): The amount of slices to split the composite up into while building it.
                          More slices will use less RAM
    """
    # Loop through each band, getting a median estimate for each
    crs = None
    transform = None

# Find each optical and cloud file in the input directory for the mgrs_tile and band combination

    mgrs_str = f'{mgrs_tile[:2]}_{mgrs_tile[2]}_{mgrs_tile[3:]}'.upper()
    band_str = band.upper()
    dates = os.listdir(in_dir)
    file_sets = []
    for date_dir in dates:
        optical_file = None
        cloud_file = None
        for file in os.listdir(os.path.join(in_dir, date_dir)):
            f_up = file.upper()
            file_path = os.path.join(in_dir, date_dir, file)
            if mgrs_str in file and band_str in file:
                optical_file = file_path
            elif mgrs_str in file and 'MSK_CLOUDS_B00' in file:
                cloud_file = file_path

        if optical_file is None or cloud_file is None:
            continue

        file_sets.append((optical_file, cloud_file))  

 # Resolve the crs and other optical file attributes (loop through all until found)
    crs = None
    transform = None
    g_nrows = None
    g_ncols = None

    for file_set in file_sets:
        with rasterio.open(file_set[0], 'r', driver='JP2OpenJPEG') as rf:
            g_nrows = rf.meta['height'] if g_nrows is None else g_nrows
            g_ncols = rf.meta['width'] if g_ncols is None else g_ncols
            crs = rf.crs if crs is None else crs
            transform = rf.transform if transform is None else transform
            if crs is not None and transform is not None and g_nrows is not None and g_ncols is not None:
                break

    if crs is None or transform is None or g_nrows is None or g_ncols is None:
        raise LookupError(f'Could not determine the following projection attributes from the available '
                          f'sentinel2 files in {in_dir}: \n' \
                          f'{"CRS" if crs is None else ""} '
                          f'{"Transform" if transform is None else ""} '
                          f'{"Number of rows" if g_nrows is None else ""} '
                          f'{"Number of columns" if g_ncols is None else ""}')

    # Determine the slicing bounds to save memory as we process
    slice_height = g_nrows / num_slices
    slice_end_pts = [int(i) for i in np.arange(0, g_nrows + slice_height, slice_height)]
    slice_bounds = [(slice_end_pts[i], slice_end_pts[i + 1] - 1) for i in range(num_slices - 1)]
    slice_bounds.append((slice_end_pts[-2], slice_end_pts[-1]))  

# Correct the images one slice at a time, and then combine the slices. 
# must create a out_dir to store the slices

    slice_dir = os.path.join(os.path.dirname(out_file), 'slices')
    os.makedirs(slice_dir, exist_ok=True)
    for k, row_bound in tqdm(enumerate(slice_bounds), desc=f'band={band}', total=num_slices, position=2):
        slice_file_path = os.path. join(slice_dir, f'{row_bound[0]}_{row_bound[1]}.tif')
        cloud_correct_imgs = []
        for file_set in tqdm(file_sets, desc=f'slice {k + 1}', leave=False, position=3):
            # Get data from files
            optical_file = file_set[0]
            cloud_file = file_set[1]
            pixels = get_img_from_file(optical_file, g_ncols, np.float32, row_bound)
            cloud_channels = get_cloud_mask_from_file(cloud_file, crs, transform, (g_nrows, g_ncols), row_bound)
            if cloud_channels is None:
                continue
            # add to list to do median filter later
            cloud_correct_imgs.append(nan_clouds(pixels, cloud_channels))
            del pixels
        corrected_stack = np.vstack([img.ravel() for img in cloud_correct_imgs])
        median_corrected = np.nanmedian(corrected_stack, axis=0, overwrite_input=True)
        median_corrected = median_corrected.reshape(cloud_correct_imgs[0].shape)
        with rasterio.open(slice_file_path, 'w', driver='GTiff', width=g_ncols, height=g_nrows,
                           count=1, crs=crs, transform=transform, dtype=np.float32) as wf:
            wf.write(median_corrected.astype(np.float32), 1)

        # release mem
        median_corrected = []
        del median_corrected
        corrected_stack = []
        del corrected_stack

# Combine slices
    with rasterio.open(out_file, 'w', driver='GTiff', width=g_ncols, height=g_nrows,
                       count=1, crs=crs, transform=transform, dtype=np.float32) as wf:
        for slice_file in os.listdir(slice_dir):
            bound_split = slice_file.split('.')[0].split('_')
            top_bound = int(bound_split[0])
            bottom_bound = int(bound_split[1])
            with rasterio.open(os.path.join(slice_dir, slice_file), 'r', driver='GTiff') as rf:
                wf.write(
                    rf.read(1),
                    window=Window.from_slices(
                        slice(top_bound, bottom_bound),
                        slice(0, g_ncols)
                    ),
                    indexes=1
                )

    shutil.rmtree(slice_dir)
    print(f'Wrote file to {out_file}')

In [ ]:
# The following codes by Lilly Jones. Alternate function to mask out clouds and water using the SCL (Scene Classification Layer) attribute of Sentinel-2 data.

# Read the SCL band, apply a mask to identify cloud and water areas, and then apply this mask to the other bands. 
# Choice depends on the quality and accuracy of the SCL band in your data, otherwise use the cloud probability method (cloud mask) by Erick Verleye above.

def mask_clouds_and_water(input_directory, output_directory):
    """
    Masks out clouds and water in Sentinel-2 data using the SCL band.

    Args:
        input_directory (str): Directory containing Sentinel-2 data.
        output_directory (str): Directory where masked data will be stored.

    Returns:
        None
    """

    # List all Sentinel-2 band files in the input directory
    band_files = glob.glob(os.path.join(input_directory, 'B*.jp2'))

    # Create output directory
    os.makedirs(output_directory, exist_ok=True)

    # Determine the SCL file path (assuming it's in the same directory)
    scl_file = os.path.join(input_directory, 'SCL.jp2')

    for band_file in band_files:
        # Define the output file path
        output_file = os.path.join(output_directory, os.path.basename(band_file))

        # Open the input band and SCL files
        ds_band = gdal.Open(band_file, gdalconst.GA_ReadOnly)
        ds_scl = gdal.Open(scl_file, gdalconst.GA_ReadOnly)

        # Read band data as a numpy array
        band_data = ds_band.ReadAsArray()

        # Read SCL data as a numpy array
        scl_data = ds_scl.ReadAsArray()

        # Create a mask for cloudy and water pixels
        # In the SCL band, value 3 corresponds to cloud and value 6 corresponds to water
        cloud_water_mask = np.logical_or(scl_data == 3, scl_data == 6)

        # Apply the mask to the band data by setting masked values to a nodata value (e.g., 0)
        band_data[cloud_water_mask] = 0

        # Create an output GeoTIFF file with the same metadata as the input band
        driver = gdal.GetDriverByName("GTiff")
        output_ds = driver.Create(output_file, ds_band.RasterXSize, ds_band.RasterYSize, 1, gdalconst.GDT_Float32)
        output_ds.SetGeoTransform(ds_band.GetGeoTransform())
        output_ds.SetProjection(ds_band.GetProjection())
        output_ds.GetRasterBand(1).WriteArray(band_data)

        # Close the datasets
        ds_band = None
        ds_scl = None
        output_ds = None

if __name__ == "__main__":
    # Define input and output directories
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the mask_clouds_and_water function
    mask_clouds_and_water(input_directory, output_directory)

In [ ]:
# VSI cloud cover

# Imports

import gdal
import os
import numpy as np
import rasterio

# Open Sentinel scene using gdal
# Set GDAL environment variables

os.environ['CPL_TMPDIR'] = '/vsimem'  # Use /vsimem for in-memory VSI
gdal.UseExceptions()

# Input Sentinel-2 image URL
input_image_path = '/vsicurl/https://your-sentinel-2-image-url'

# Open the Sentinel-2 image using GDAL
dataset = gdal.Open(input_image_path)

# Define a cloud detection function
def cloud_detection(band_red, band_swir, threshold=0.2):
    # Calculate a cloud probability map using custom logic
    cloud_prob = band_swir / (band_red + band_swir + 1e-10)

    # Set cloudy pixels to 1 and non-cloudy pixels to 0
    cloud_mask = cloud_prob > threshold

    return cloud_mask

# Read red and SWIR bands using rasterio
with rasterio.open(input_image_path) as src:
    red_band = src.read(3)
    swir_band = src.read(11)

# Simple algorithm for cloud detection using a threshold
cloud_mask = cloud_detection(red_band, swir_band)

# Apply the cloud mask to the scene (sets the pixel values to 0 for cloudy areas based on the cloud mask)

for i in range(1, dataset.RasterCount + 1):
    band = dataset.GetRasterBand(i)
    data = band.ReadAsArray()
    data[cloud_mask] = 0  # Set cloudy pixels to 0
    band.WriteArray(data)

# Save the output

output_image_path = '/vsimem/cloud_corrected_sentinel2.tif'  # Output VSI path
dataset.FlushCache()
dataset = None

# Create a copy of the corrected image with the VSI path

gdal.Translate(output_image_path, input_image_path)

In [ ]:
# Function to extract spectra from Sentinel-2 data and resample relevant bands

def extract_spectra_and_resample(xml_file, geotiff_file, output_csv_file):
    try:
        # Parse the Sentinel-2 XML file to extract band information
        tree = ET.parse(xml_file)
        root = tree.getroot()

        bands_info = []
        for elem in root.iterfind(".//BANDS/BAND"):
            band_name = elem.find("BAND_NAME").text
            band_center_wavelength = float(elem.find("CENTRAL_WAVELENGTH").text)
            bands_info.append((band_name, band_center_wavelength))

        # Open the geotiff file using rasterio
        with rasterio.open(geotiff_file) as src:
            band_count = src.count
            band_data = []

            # Iterate over each band and read data
            for band_index in range(1, band_count + 1):
                band_data.append(src.read(band_index))

        # Resample the 20m resolution bands to 10m resolution using the average method
        resampled_band_data = []
        resample_indices = [1, 2, 3, 4, 8, 11, 12]  # Indices of 20m bands to be resampled
        for i in range(len(band_data)):
            if i in resample_indices:
                resampled_band = (band_data[i] + band_data[i + 1]) / 2.0
                resampled_band_data.append(resampled_band)
            elif i == 0 or i == 5:  # Bands 2 and 8 are already 10m resolution
                resampled_band_data.append(band_data[i])

        # Extract reflectance data from each band and create a dataframe
        spectra_data = {band_name: resampled_band_data[i] for i, (band_name, _) in enumerate(bands_info)}
        df = pd.DataFrame(spectra_data)

        # Add the center wavelength information to the dataframe
        df['Wavelength (micrometers)'] = [wavelength for (_, wavelength) in bands_info]

        # Rename columns for samples
        df.columns = [f'Sample_{i}' for i in range(df.shape[1] - 1)] + ['Wavelength (micrometers)']

        # Display the dataframe (Optional: comment out if you have many samples)
        print(df)

        # Save the dataframe to a CSV file
        df.to_csv(output_csv_file, index=False)

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    sentinel2_xml_file = "path/to/sentinel2.xml"
    sentinel2_geotiff_file = "path/to/sentinel2_geotiff.tif"
    output_csv_file = "spectra_table.csv"
    
    # Extract spectra and resample relevant bands
    extract_spectra_and_resample(sentinel2_xml_file, sentinel2_geotiff_file, output_csv_file)

In [ ]:
# Calculate the Normalized Difference Vegetation Index (NDVI) from Sentinel-2 bands 4 (red) and 8 (nir). NDVI is calculated as (NIR - Red) / (NIR + Red).

def calculate_ndvi(input_directory, output_directory):
    """
    Calculates NDVI from Sentinel-2 bands 4 (red) and 8 (NIR).

    Args:
        input_directory (str): Directory containing Sentinel-2 data.
        output_directory (str): Directory where NDVI data will be stored.

    Returns:
        None
    """

    # List all Sentinel-2 band files in the input directory
    band4_file = glob.glob(os.path.join(input_directory, 'B04.jp2'))[0]  # Red band
    band8_file = glob.glob(os.path.join(input_directory, 'B08.jp2'))[0]  # NIR band

    # Create output directory
    os.makedirs(output_directory, exist_ok=True)

    # Initialize variables to store band data
    red_band = None
    nir_band = None

    for band_file in [band4_file, band8_file]:
        # Define the output file path
        output_file = os.path.join(output_directory, os.path.basename(band_file).replace('B0', 'NDVI'))

        # Open the input band file
        ds_band = gdal.Open(band_file, gdalconst.GA_ReadOnly)

        # Read band data as a numpy array
        band_data = ds_band.ReadAsArray()

        # Assign band data to respective variables
        if 'B04' in band_file:
            red_band = band_data.astype(np.float32)
        elif 'B08' in band_file:
            nir_band = band_data.astype(np.float32)

        # Close the dataset
        ds_band = None

    # Check if both bands are available
    if red_band is not None and nir_band is not None:
        # Calculate NDVI
        ndvi = (nir_band - red_band) / (nir_band + red_band)

        # Create an output GeoTIFF file with the same metadata as the input band
        driver = gdal.GetDriverByName("GTiff")
        output_ds = driver.Create(output_file, ds_band.RasterXSize, ds_band.RasterYSize, 1, gdalconst.GDT_Float32)
        output_ds.SetGeoTransform(ds_band.GetGeoTransform())
        output_ds.SetProjection(ds_band.GetProjection())
        output_ds.GetRasterBand(1).WriteArray(ndvi)

        # Close the output dataset
        output_ds = None
    else:
        print("Error: Both red and NIR bands are required to calculate NDVI.")

if __name__ == "__main__":
    # Define input and output directories
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the calculate_ndvi function
    calculate_ndvi(input_directory, output_directory)

In [ ]:
# calculate mean NDVI for all time-steps


def calculate_ndvi(input_directory, output_directory):
    """
    Calculates NDVI from Sentinel-2 bands 4 (red) and 8 (NIR).

    Args:
        input_directory (str): Directory containing Sentinel-2 data for multiple time-steps.
        output_directory (str): Directory where mean NDVI data will be stored.

    Returns:
        None
    """
    # List all time-steps (subdirectories) in the input directory
    time_steps = [d for d in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, d))]

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Initialize arrays to store NDVI values and count of valid pixels
    ndvi_sum = None
    valid_pixel_count = None

    for time_step in time_steps:
        # Get the directory containing Sentinel-2 data for the current time-step
        time_step_directory = os.path.join(input_directory, time_step)

        # List all Sentinel-2 band files in the current time-step directory
        band4_file = glob.glob(os.path.join(time_step_directory, 'B04.jp2'))[0]  # Red band
        band8_file = glob.glob(os.path.join(time_step_directory, 'B08.jp2'))[0]  # NIR band

        for band_file in [band4_file, band8_file]:
            # Open the input band file
            ds_band = gdal.Open(band_file, gdalconst.GA_ReadOnly)

            # Read band data as a numpy array
            band_data = ds_band.ReadAsArray()

            # Calculate NDVI
            if 'B04' in band_file:
                red_band = band_data.astype(np.float32)
            elif 'B08' in band_file:
                nir_band = band_data.astype(np.float32)

        # Calculate NDVI for the current time-step
        ndvi = (nir_band - red_band) / (nir_band + red_band)

        # Initialize the arrays for NDVI sum and valid pixel count
        if ndvi_sum is None:
            ndvi_sum = np.zeros_like(ndvi, dtype=np.float64)
            valid_pixel_count = np.zeros_like(ndvi, dtype=np.int64)

        # Update the sum of NDVI values and valid pixel count
        ndvi_sum += ndvi
        valid_pixel_count += 1

    # Calculate mean NDVI by dividing the sum by the valid pixel count
    mean_ndvi = np.divide(ndvi_sum, valid_pixel_count, where=valid_pixel_count != 0)

    # Get the metadata (geotransform and projection) from one of the input bands
    ds_band = gdal.Open(band4_file, gdalconst.GA_ReadOnly)
    geotransform = ds_band.GetGeoTransform()
    projection = ds_band.GetProjection()
    ds_band = None

    # Create an output GeoTIFF file with the same metadata as the input band
    output_file = os.path.join(output_directory, 'mean_ndvi.tif')
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(output_file, mean_ndvi.shape[1], mean_ndvi.shape[0], 1, gdalconst.GDT_Float32)
    output_ds.SetGeoTransform(geotransform)
    output_ds.SetProjection(projection)
    output_ds.GetRasterBand(1).WriteArray(mean_ndvi)

    # Close the output dataset
    output_ds = None

if __name__ == "__main__":
    # Define input and output directories
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data for multiple time-steps
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the calculate_ndvi function
    calculate_ndvi(input_directory, output_directory)

In [ ]:

# plot mean NDVI


def plot_mean_ndvi(ndvi_file):
    """
    Plots the mean NDVI from a GeoTIFF file.

    Args:
        ndvi_file (str): Path to the GeoTIFF file containing mean NDVI data.

    Returns:
        None
    """
    # Open the GeoTIFF file using rasterio
    with rasterio.open(ndvi_file) as src:
        # Read the NDVI data as a numpy array
        ndvi_data = src.read(1)

        # Create a colormap for NDVI (green to red)
        cmap = plt.cm.RdYlGn

        # Set nodata values to transparent
        cmap.set_bad(alpha=0)

        # Create a plot
        plt.figure(figsize=(10, 10))
        plt.imshow(ndvi_data, cmap=cmap, vmin=-1, vmax=1)
        plt.colorbar(label='NDVI')
        plt.title('Mean NDVI')
        plt.axis('off')

        # Show the plot
        plt.show()

if __name__ == "__main__":
    # Define the path to the mean NDVI GeoTIFF file
    ndvi_file = "path_to_mean_ndvi.tif"  # Replace with the actual file path

    # Call the plot_mean_ndvi function
    plot_mean_ndvi(ndvi_file)

In [ ]:
# plot spectral signature for all bands reads and plots the pixel values for each band


def plot_spectral_signature(input_directory):
    """
    Plots the spectral signature from all bands of Sentinel-2 data.

    Args:
        input_directory (str): Directory containing Sentinel-2 data.

    Returns:
        None
    """
    # List all Sentinel-2 band files in the input directory
    band_files = sorted(glob.glob(os.path.join(input_directory, 'B*.jp2')))

    # Initialize an empty list to store band names
    band_names = []

    # Initialize empty arrays to store spectral data for each band
    spectral_data = []

    # Open and read data for each band
    for band_file in band_files:
        # Get the band name from the file name (e.g., B02.jp2)
        band_name = os.path.splitext(os.path.basename(band_file))[0]
        band_names.append(band_name)

        # Open the band file using GDAL
        ds_band = gdal.Open(band_file, gdal.GA_ReadOnly)

        # Read the band data as a numpy array
        band_data = ds_band.ReadAsArray()

        # Append the band data to the spectral_data list
        spectral_data.append(band_data)

    # Create a plot for the spectral signature
    plt.figure(figsize=(12, 6))
    for band_name, band_data in zip(band_names, spectral_data):
        plt.plot(band_data, label=band_name)

    # Set plot labels and title
    plt.xlabel('Pixel Value')
    plt.ylabel('Reflectance')
    plt.title('Spectral Signature of Sentinel-2 Bands')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()

if __name__ == "__main__":
    # Define input directory containing Sentinel-2 data
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data

    # Call the plot_spectral_signature function
    plot_spectral_signature(input_directory)

In [ ]:
# Create spectral libraries
# Collect spectra across seasons and years (> 50 per class)
# Classes: each vegetation type's green vegetation and dry vegetation, bare soil, water and shade
# Collect spectra at the pixel level (capture the variability within classes, sunlight pixels preferentially - we will have the shade class)
# NEON R code to be translated to Python: https://github.com/earthlab/neonhs
# May use Python functions from this: https://www.spectralpython.net/

# 4. Create metadata
# A final, merged spectral library must have a metadata (.csv) with at least spectrum ID (column 1), class (2), and sub-class (3) description. 
# For the vegetation type classes the order is: 1. class: green vegetation or non-photosynthetic vegetation (i.e. dry matter); 2. sub-class: vegetation type. 
# For the other classes (bare # # soil, water..), repeat the class content in the sub-class column.

# 5. Endmember Selection - WIP

# 6. Multiple Endmber Spectral Mixture Analysis Emulation WIP

# 7. Dry matter Index Calculation WIP

In [ ]:
# export output to PetaLibrary 

#!/bin/bash

# Variables
local_file="/path/to/local/file.zip"
remote_username="your_username"
remote_host="petalibrary.colorado.edu"
remote_directory="/path/to/remote/directory/"

# Copy file using SCP
scp "$local_file" "$remote_username@$remote_host:$remote_directory"

echo "File transferred successfully."

bash transfer_script.sh

In [ ]:
# Attempts to use vsicurl and STAC

import geopandas as gpd
import osmdata
from rasterio.plot import show
from rasterio.windows import from_bounds
import boto3
from io import BytesIO
import subprocess
import json
from zipfile import ZipFile
import pandas as pd
import matplotlib.pyplot as plt

# AWS credentials and bucket name
aws_access_key_id = 'YOUR_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_SECRET_ACCESS_KEY'
bucket_name = 'your-bucket-name'

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Load and filter shapefiles
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world[world['name'] == 'United States']
california = usa[usa['name'] == 'California']  # Filter for California

# Read and filter GMW shapefiles
base_url = "https://datadownload-production.s3.amazonaws.com"
y2017 = "GMW_v3_2017.zip"
y2020 = "GMW_v3_2020.zip"

gmw2017_california = gpd.read_file(f"{base_url}/{y2017}", vfs="zip://")
gmw2017_california = gmw2017_california[gmw2017_california.intersects(california.unary_union)]

gmw2020_california = gpd.read_file(f"{base_url}/{y2020}", vfs="zip://")
gmw2020_california = gmw2020_california[gmw2020_california.intersects(california.unary_union)]

# Plot the filtered shapefiles
gmw2017_california.plot()
gmw2020_california.plot()

# Define a bounding box for California and transform it to different coordinate systems
bbox = osmdata.getbb("California, USA")
bbox_4326 = bbox.to_crs(epsg=4326)
bbox_32618 = bbox.to_crs(epsg=32618)

def lambda_handler(event, context):
    # Replace 'YOUR_SENTINEL_API_KEY' with your actual Sentinel Hub API key
    sentinel_api_key = 'YOUR_SENTINEL_API_KEY'
    
    # Get the input parameters from the Step Function event
    bbox = event['bbox']
    time_range = event['time_range']
    
    # Call Sentinel Hub API to download Sentinel-2 data
    download_url = f'https://services.sentinel-hub.com/api/v1/process/download'
    request_payload = {
        'input': {
            'bounds': {
                'geometry': {
                    'type': 'Polygon',
                    'coordinates': [bbox]
                }
            },
            'data': [{
                'type': 'S2L1C',
                'dataFilter': {
                    'timeRange': time_range,
                    'maxCloudCoverage': 10
                }
            }]
        },
        'output': {
            'width': 512,
            'height': 512,
            'responses': [{
                'identifier': 'default',
                'format': {
                    'type': 'image/tiff'
                }
            }]
        }
    }
    
    response = requests.post(download_url, json=request_payload, headers={'Authorization': f'Bearer {sentinel_api_key}'})
    
    # Process the downloaded data 
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('/tmp')  # Extract the contents to /tmp directory

In [ ]:
    
# VSI processing steps

import subprocess
import json
from io import BytesIO
from zipfile import ZipFile

def lambda_handler(event, context):
    # Replace 'YOUR_STAC_API_URL' with the actual STAC API endpoint URL
    stac_api_url = 'YOUR_STAC_API_URL'
    
    # Get the input parameters from the Step Function event
    # For example, you might pass the bounding box coordinates, time range, etc.
    bbox = event['bbox']
    time_range = event['time_range']
    
    # Use STAC API to search for Sentinel-2 data
    search_payload = {
        'bbox': bbox,
        'time': time_range,
        'collections': ['sentinel-2-l1c']
    }
    
    search_url = f'{stac_api_url}/search'
    search_response = subprocess.run(['vsi', 'curl', '-X', 'POST', '-d', json.dumps(search_payload), search_url], capture_output=True, text=True)
    
    # Parse the search response to get the download URL
    download_url = json.loads(search_response.stdout)['features'][0]['assets']['data']['href']
    
    # Download Sentinel-2 data using vsi curl
    download_response = subprocess.run(['vsi', 'curl', download_url], capture_output=True, text=True)
    
    # Process the downloaded data (you might want to use a more sophisticated method)
    with ZipFile(BytesIO(download_response.stdout.encode())) as zip_file:
        zip_file.extractall('/tmp')  # Extract the contents to /tmp directory
    
    # Your VSI processing steps here (replace with actual processing logic)
    processed_data = b'Your processed data'
    
    return {
        'statusCode': 200,
        'body': processed_data
    }
   

# Step Function definition
{
  "Comment": "Sentinel-2 Data Processing",
  "StartAt": "DownloadAndProcess",
  "States": {
    "DownloadAndProcess": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:REGION:ACCOUNT_ID:function:YourLambdaFunctionName",
      "End": true
    }
  }
}

# Save the processed data back to S3
processed_data = b'Your processed data in the cloud'  # Replace this with your actual processed data
with BytesIO(processed_data) as processed_data_buffer:
    s3.upload_fileobj(processed_data_buffer, bucket_name, 'path/to/processed_data.tif')

# Data Visualization 
data = pd.read_csv('path/to/processed_data.csv') 
plt.figure(figsize=(10, 5))
plt.plot(data['wavelength_nm'], data['reflectance'])
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.title('Reflectance vs. Wavelength')
plt.savefig('reflectance_plot.png')

# Upload the visualization to S3
with open('reflectance_plot.png', 'rb') as plot_file:
    s3.upload_fileobj(plot_file, bucket_name, 'path/to/reflectance_plot.png')

In [ ]:
# use output shapefile in ArcGIS with arcpy

import arcpy

# Set the workspace where you want to save the output
arcpy.env.workspace = r'C:\Path\To\Output\Directory'

# Create a feature class to save your data
output_feature_class = "OutputData.shp"  # Change the name as needed

# Create a new feature class (shapefile)
arcpy.CreateFeatureclass_management(arcpy.env.workspace, output_feature_class, "POINT")

# Open an insert cursor to add points to the feature class
cursor = arcpy.da.InsertCursor(output_feature_class, ["SHAPE@XY"])

# Loop through your data and add points to the feature class
for point in data:
    cursor.insertRow([point])

# Clean up
del cursor

print(f"Data has been saved to {output_feature_class}")